In [ ]:
import scipy
import sys
import pandas as pd
import numpy as np
from lxml import etree
import FeatureExtraction as FE
from sklearn.cluster import KMeans
sys.path.insert(0, '../notebooks/libs/')

In [ ]:
# ucitavanje corpusa i ID-eva predatora iz prethodne faze
pathToFile='../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'
#pathToIds = '../../dataset/test/pan12-sexual-predator-identification-groundtruth-problem1.txt'
#predators_ids = FE.sexual_predator_ids(pathToIds)
pathToIds = '../../notebooks/models/classified_preadtors_822.txt'
tree=etree.parse(pathToFile)

#ucitaj ids predatora iz datoteke
f = open(pathToIds,'r')
predators_ids = f.read().split('\n')[:-1]

In [ ]:
# !!!PAZI! Izvodi samo ako zelis nac vokabular iz najboljih znacajki SVM-a 
training_xml = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_xml = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

sexual_predator_ids_file = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'

chat_based_features_csv_train='../../csv/chat_based_features_training.csv'
chat_based_features_csv_test='../../csv/chat_based_features_test.csv'

In [ ]:
# !!!PAZI! Izvodi samo ako zelis nac vokabular iz najboljih znacajki SVM-a  
minimal_number_of_messages_treshold=5
train_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_train))
test_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_test))
train_frame=train_frame[train_frame['number of messages sent'] >=minimal_number_of_messages_treshold]
test_frame=test_frame[test_frame['number of messages sent']>=minimal_number_of_messages_treshold]
y_train=np.ravel(train_frame[[-1]])
y_test=np.ravel(test_frame[[-1]])

In [ ]:
# !!!PAZI! Izvodi samo ako zelis nac vokabular iz najboljih znacajki SVM-a 
# OVO TRAJE STOLJECIMA. Izvodi samo ako bas trebas :) 
documents_training=FE.prepare_for_tf_idf(training_xml,False,True,minimal_number_of_messages_treshold)
document_testing=FE.prepare_for_tf_idf(test_xml,False,True,minimal_number_of_messages_treshold)

In [ ]:
# !!!PAZI! Izvodi samo ako zelis nac vokabular iz najboljih znacajki SVM-a 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english',min_df=10,max_df=0.5)
matrix_training=tfidf.fit_transform(documents_training)
matrix_testing=tfidf.transform(document_testing)

In [ ]:
# !!!PAZI! Izvodi samo ako zelis nac vokabular iz najboljih znacajki SVM-a 
from sklearn import svm
classifier=svm.SVC(C=0.7454,kernel='linear',max_iter=100)
classifier.fit(matrix_training,y_train)

In [ ]:
# !!!PAZI! Izvodi samo ako zelis nac vokabular iz najboljih znacajki SVM-a 
# vrati vokabular iz najboljih znacajki SVM-a kao listu
max_features=500
sortedIndices=np.array(np.argsort(abs(classifier.coef_.todense())))
k=list(sortedIndices[0])
ks=k[len(k)-max_features:]
names=np.array(tfidf.get_feature_names())
voc= names[ks]
lll= list(voc)

In [ ]:
# path_to_file - put do xml datoteke
# predator_ids - lista id seksualnih predatora
# returns: conv_ids - ids konverzacija, line_nrs - brojevi linija, line_texts - sadrzaj linija teksta
def prepare_data_for_clustering(path_to_file,predators_ids):
    tree = etree.parse(pathToFile)
    dictionary = FE.extract_author_conversation_node_dictionary_from_XML(tree)
    #lista konverzacija u kojima sudjeluje predator
    conversations = []
    conv_ids = []
    line_nrs = []
    line_texts = []
    for predator in predators_ids:
        convs = dictionary[predator]
        for c in convs:
            conversations.append(c)
    for conv_node in conversations:
        conv_id  = conv_node.attrib["id"]
        messgs = conv_node.xpath("./message")
        for mess_node in messgs:          
            text = mess_node.xpath("./text")[0].text
            if text is None:
                continue
            if len(text.split(" "))<2:
                continue
            conv_ids.append(conv_id)
            line_nr = mess_node.attrib["line"]
            line_nrs.append(line_nr)
            line_texts.append(text)
    return conv_ids,line_nrs,line_texts

In [ ]:
# priprema podataka za clustering 
conv_ids,line_nrs,line_texts = prepare_data_for_clustering(pathToFile,predators_ids)

In [ ]:
# funkcija stvara tf-idf matricu na temelju odabranog vokabulara
from sklearn.feature_extraction.text import TfidfVectorizer
def make_tf_idf_matrix_for_clustering_from_XML(line_texts,vocab):
    tfidf=TfidfVectorizer(stop_words='english',min_df=10,vocabulary = vocab)
    return tfidf.fit_transform(line_texts), tfidf.get_feature_names()

In [ ]:
# odabir vokabulara iz datoteke
path_to_vocab = 'profanity.txt'
f = open(path_to_vocab,'r')
prof=set(f.read().split('\n'))

In [ ]:
# stvaranje tf-idf matrice, na temelju odabranog vokabulara
matrix, ft_names = make_tf_idf_matrix_for_clustering_from_XML(line_texts,lll)
matrix.shape

In [ ]:
# CLUSTERING: k - means algoritam, 2 klase
random_state = 42
y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(matrix)

In [ ]:
#ako je flag true, u file ce se ispisati i sadrzaj recenica
#ako je flag false, file je u formatu id konverzacije, broj linije
import codecs
def dump_prediction_to_file(prediction_vector,conv_ids,line_nrs,line_texts,show_text=True):
    f = codecs.open('../../dataset/class_zero.txt','w','utf-8')
    g = codecs.open('../../dataset/class_one.txt','w','utf-8')
    for i,p in enumerate(prediction_vector):
        if p==0:
            line = str(conv_ids[i])+"\t"+str(line_nrs[i])
            if show_text:
                line+="\t"+line_texts[i]+"\n"
            else:
                line+="\n"
            f.write(line)
        else:
            line = str(conv_ids[i])+"\t"+str(line_nrs[i])
            if show_text:
                line+="\t"+line_texts[i]+"\n"
            else:
                line+="\n"
            g.write(line)
            
    f.close()
    g.close()
    

In [ ]:
#upis rezultata clusteringa u datoteku
dump_prediction_to_file(y_pred,conv_ids,line_nrs,line_texts,show_text=True)

In [ ]:
def get_clustering_results_as_set(pathToFile):
    f = open(pathToFile,'r')
    elements = f.read().split('\n')
    return set(elements)

In [ ]:
def pronadi_presjek(s1,s2):
    return len(s1.intersection(s2)),len(s1.difference(s2))

In [ ]:
# putanje do datoteka s rezultatima i groundtruth problema
path1 = '../../dataset/class_zero.txt'
path2 = '../../dataset/class_one.txt'
path3 = '../../dataset/test/pan12-sexual-predator-identification-groundtruth-problem2.txt'

In [ ]:
# spremanje rezultata iz datoteka u setove
s1 = get_clustering_results_as_set(path1)
s2 = get_clustering_results_as_set(path2)
s3 = get_clustering_results_as_set(path3)

In [ ]:
# pronalazak presjeka i razlike s groundtruthom za oba clustera
p1,r1 = pronadi_presjek(s1,s3)
p2,r2 = pronadi_presjek(s2,s3)

# ispis rezultata
print "presjek0: " +str(p1)
print "presjek1: " +str(p2)
print "razlika0: "+str(r1)
print "razlika1: "+str(r2)

In [ ]:
# recall = koliko smo detektirali/kolko smo ih trebali detektirati
# number_of_suggestive_lines = len(s3)

#detektirano u clusteru 0 = p1
recall_1 = (1.0*p1)/len(s3)
print "recall 1: " + str(recall_1)

#detektirano u clusteru 1 = p2
recall_2 = (1.0*p2)/len(s3) 
print "recall 2: " + str(recall_2)

In [ ]:
# precision = kolko smo detektirali/duljina naseg clustera
precision_1 = (1.0*p1)/len(s1)
print "precision 1: "+str(precision_1)

precision_2 = (1.0*p2)/len(s2)
print "precision 2: "+str(precision_2)

In [ ]:
#f-measure
def f_measure(p,r,beta):
    f = (1+beta*beta)*(p*r)
    f = (1.0)*f/((beta*beta*p)+r)
    return f

In [ ]:
# rezultati F3 - mjere za oba clustera
f3_1 = f_measure(precision_1,recall_1,3)
print "1) f3: "+str(f3_1)

f3_2 = f_measure(precision_2,recall_2,3)
print "2) f3: "+str(f3_2)

Ispitivanja različitih vokabulara.

In [ ]:
path_to_vocab = 'negative.txt'
f = open(path_to_vocab,'r')
neg=set(f.read().split('\n'))

In [ ]:
path_to_vocab = 'profanity.txt'
f = open(path_to_vocab,'r')
prof=set(f.read().split('\n'))

In [ ]:
final_vocab = neg.union(prof)

Konkatenacija SVM i prostog vokabulara.

In [ ]:
final_vocab = set(lll).union(neg).union(prof)

In [ ]:
matrix, ft_names = make_tf_idf_matrix_for_clustering_from_XML(line_texts,final_vocab)
matrix.shape

In [ ]:
s2.intersection(s3)